In [4]:
import numpy as np
from tqdm.notebook import tqdm
import glob

import xarray as xr
import seaborn as sns;
import netCDF4 as nc
import os
from netCDF4 import Dataset


## Because the desired output is one slice/ image/grid for each year, I will process year by year
def continuous_meet(cond, window_size, dim):
    """
    Continuously meet a given condition along a dimension.
    """
    _found = cond.rolling(dim={'time': window_size},
                          center=True).sum(skipna=True).fillna(False).astype(np.float)

    detected = np.array(
        _found.rolling(dim={'time': window_size})
        .reduce(lambda a, axis: (a == window_size).any(axis=axis))
        .fillna(False)
        .astype(bool)
    )

    indices = (detected * np.arange(detected.shape[0]).reshape(detected.shape[0], 1, 1))
    indices[indices == 0] = detected.shape[0]
    output = indices.argmin(axis=0)

    return xr.DataArray(output)

# sub_data = xr.open_dataset('y12_break5d_30.nc')
# print(sub_data)
# ice =  sub_data.__xarray_dataarray_variable__
# ice.plot()

data = xr.open_dataset('daily_sic_subset.nc',engine ="netcdf4") # opening sea ice concentration file 
# ## Arange months up to Aug to December ##
def is_months(month):
    return (month >= 8) & (month <= 12)

In [5]:
data = data.sel(time=is_months(data['time.month']))

In [6]:
# ## Select SIC variable ##
sic_xr = data.SI_12km_SH_ICECON_DAY_SpPolarGrid12km
latitude = data.GridLat_SpPolarGrid12km
longitude = data.GridLon_SpPolarGrid12km
time = data.time

In [10]:
sic_xr.projection.isel(s_rho=-1, ocean_time=0).plot(x="GridLon_SpPolarGrid12km", y="GridLat_SpPolarGrid12km")

AttributeError: 'str' object has no attribute 'isel'

In [7]:
sic_xr

<xarray.DataArray 'SI_12km_SH_ICECON_DAY_SpPolarGrid12km' (time: 150,
                                                           YDim_SpPolarGrid12km: 664,
                                                           XDim_SpPolarGrid12km: 632)>
[62947200 values with dtype=int32]
Coordinates:
  * time                     (time) datetime64[ns] 2012-08-01 ... 2014-08-28
    GridLon_SpPolarGrid12km  (YDim_SpPolarGrid12km, XDim_SpPolarGrid12km) float64 ...
    GridLat_SpPolarGrid12km  (YDim_SpPolarGrid12km, XDim_SpPolarGrid12km) float64 ...
Dimensions without coordinates: YDim_SpPolarGrid12km, XDim_SpPolarGrid12km
Attributes:
    long_name:   Sea ice concentration daily average
    units:       percent
    projection:  Polar Stereographic
    comment:     data value meaning: 0 -- Open Water, 110 -- missing/not calc...

In [1]:
from mpl_toolkits.basemap import Basemap

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

In [2]:
# https://stackoverflow.com/questions/13796315/plot-only-on-continent-in-matplotlib

In [3]:
# https://basemaptutorial.readthedocs.io/en/latest/utilities.html

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from osgeo import gdal
from numpy import linspace
from numpy import meshgrid

map = Basemap(projection='tmerc', 
              lat_0=0, lon_0=3,
              llcrnrlon=1.819757266426611, 
              llcrnrlat=41.583851612359275, 
              urcrnrlon=1.841589961763497, 
              urcrnrlat=41.598674173123)

ds = gdal.Open("../sample_files/dem.tiff")
data = ds.ReadAsArray()

x = linspace(0, map.urcrnrx, data.shape[1])
y = linspace(0, map.urcrnry, data.shape[0])

xx, yy = meshgrid(x, y)

cmap = plt.get_cmap('PiYG')

colormesh = map.pcolormesh(xx, yy, data, vmin = 500, vmax = 1300, cmap=cmap)
contour = map.contour(xx, yy, data, range(500, 1350, 50), colors = 'k', linestyles = 'solid')

map.colorbar(colormesh)
cb = map.colorbar(colormesh, location='bottom', label="contour lines")

cb.add_lines(contour)
cb.set_ticks([600, 760, 1030, 1210])

plt.show()

In [ ]:
sic_xr.shape

In [ ]:
X = sic_xr

In [ ]:
X = X.values

In [ ]:
for i in tqdm(range(len(X))):
    for j in range(len(X[i])):
        if sum(X[i][j]) == 0:
#             print('Yes')
            X[i][j] = 140

In [ ]:
# ## Here we want all the values that are over 100 (missing or land mask) to be set to nan ##

# sic_xr = sic_xr.where(sic_xr < 101,130)
# sic_xr = sic_xr.where(sic_xr == 0,140)

In [ ]:
# latitude = data.GridLat_SpPolarGrid12km
# longitude = data.GridLon_SpPolarGrid12km
# time = data.time

In [ ]:
sic_xr[0][0]

In [ ]:
# data_xr = xr.DataArray(X, coords={'y': latitude,'x': longitude,'time': time}, 
# dims=["YDim_SpPolarGrid12km", "XDim_SpPolarGrid12km", "time"])

In [ ]:
c_data = continuous_meet(sic_xr<30, window_size=3, dim='time')

In [ ]:
c_data.plot()

In [ ]:
c_data = continuous_meet(sic_xr, window_size=3, dim='time')

In [ ]:
c_data.plot()

In [ ]:
cond = sic_xr<30 
window_size=3
dim='time'

In [ ]:
 _found = cond.rolling(dim={'time': window_size},
                          center=True).sum(skipna=True).fillna(False).astype(np.float)

detected = np.array(
    _found.rolling(dim={'time': window_size})
    .reduce(lambda a, axis: (a == window_size).any(axis=axis))
    .fillna(False)
    .astype(bool)
)

indices = (detected * np.arange(detected.shape[0]).reshape(detected.shape[0], 1, 1))
# indices[indices == 0] = detected.shape[0]
# output = indices.argmin(axis=0)


In [ ]:
_found

In [ ]:
x = sic_xr.values

In [ ]:
range(len(x[0]))

In [ ]:
sic_xr[0][0]

In [ ]:
sic_xr[0][0]<30

In [ ]:
for i in tqdm(range(len(sic_xr))):
    for j in range(len(sic_xr[i])):
#         print(i,j)
        if all(sic_xr[i][j])>=30:
            print(i,j)

In [ ]:
x[0][0]

In [ ]:
for i in range(len(x[0])):
#     if sum(x[0][i])!=632:
    print(i)

In [ ]:
sum(x[0][0])

In [ ]:
sum([sum(i[0]) for i in x[0]])

In [ ]:
sum(x[0])

In [ ]:
for j in range(50):
    for i in range(50):
        sic_xr[j][i] = 140

In [ ]:
sic_xr[0][2]

In [ ]:
sic_xr = sic_xr.where(sic_xr == 0,-999)

In [ ]:
sic_xr